# Assignment: Data Wrangling
### `! git clone https://github.com/ds4e/wrangling`
### Do Q1 and Q2, and either Q3 or Q4, for a total of 3 questions.

**Q1.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the Minnesota police use of for data, `./data/mn_police_use_of_force.csv`, clean the `subject_injury` variable, handling the NA's; this gives a value `Yes` when a person was injured by police, and `No` when no injury occurred. What proportion of the values are missing? Is this a concern? Cross-tabulate your cleaned `subject_injury` variable with the `force_type` variable. Are there any patterns regarding when the data are missing?
3. Dummy variable: For the pretrial data covered in the lecture, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

**Numeric Variable**

In [2]:
airbnburl = 'https://raw.githubusercontent.com/aileenkent/wrangling/refs/heads/main/assignment/data/airbnb_hw.csv'
airbnb = pd.read_csv(airbnburl)

In [3]:
airbnb.head() #to figure out what the data looks like

,Host Id,Host Since,Name,Neighbourhood,Property Type,Review Scores Rating (bin),Room Type,Zipcode,Beds,Number of Records,Number Of Reviews,Price,Review Scores Rating
0,5162530,NaN,1 Bedroom in Prime Williamsburg,Brooklyn,Apartment,NaN,Entire home/apt,11249.0,1.0,1,0,145,NaN
1,33134899,NaN,"Sunny, Private room in Bushwick",Brooklyn,Apartment,NaN,Private room,11206.0,1.0,1,1,37,NaN
2,39608626,NaN,Sunny Room in Harlem,Manhattan,Apartment,NaN,Private room,10032.0,1.0,1,1,28,NaN
3,500,6/26/2008,Gorgeous 1 BR with Private Balcony,Manhattan,Apartment,NaN,Entire home/apt,10024.0,3.0,1,0,199,NaN
4,500,6/26/2008,Trendy Times Square Loft,Manhattan,Apartment,95.0,Private room,10036.0,3.0,1,39,549,96.0


In [4]:
airbnb['Price'].describe() #understanding what might be some issues with the data

,Price
count,30478
unique,511
top,150
freq,1481


In [5]:
airbnb['Price'].isnull().sum() #checking for any missing, null, or nan values that are not coded as 0

0

In [6]:
airbnb['Price'] = airbnb['Price'].str.replace(',', '')
airbnb['Price'] = airbnb['Price'].str.replace('$', '')
airbnb['Price'] = airbnb['Price'].str.replace(' ', '') # removing common symbols in number based data that prevents the transistion to integers
airbnb['Price'] = airbnb['Price'].astype(int) # converting from object type class so can run math things on it

In [8]:
airbnb['Price'].describe()

,Price
count,30478.000000
mean,163.589737
std,197.785454
min,10.000000
25%,80.000000
50%,125.000000
75%,195.000000
max,10000.000000


**Categorical Variable**

In [9]:
MN_police_url = 'https://raw.githubusercontent.com/aileenkent/wrangling/refs/heads/main/assignment/data/mn_police_use_of_force.csv'
MN_police_data = pd.read_csv(MN_police_url)

In [10]:
MN_police_data['subject_injury'].isnull().sum()

9848

In [11]:
MN_police_data['subject_injury'].value_counts()

,count
subject_injury,
Yes,1631
No,1446


In [13]:
proportion_missing = MN_police_data['subject_injury'].isnull().sum() / len(MN_police_data['subject_injury'])
proportion_missing #figuring out how big of a problem the missing values are going to be

0.7619342359767892

In [21]:
MN_police_data['subject_injury'] = MN_police_data['subject_injury'].fillna('Not recorded') #creating a new option because simply putting no for 76% of the data seems like might cause some overlooking

In [22]:
MN_police_data['subject_injury'].isnull().sum() #making sure the replacement work

0

In [14]:
MN_police_data['force_type'].isnull().sum()

0

In [15]:
print(MN_police_data['force_type'].unique())

['Bodily Force' 'Chemical Irritant' 'Taser' 'Improvised Weapon'
 'Gun Point Display' 'Police K9 Bite' 'Baton' 'Firearm'
 'Less Lethal Projectile' 'Maximal Restraint Technique' 'Less Lethal']


In [23]:
crossing = pd.crosstab(MN_police_data['subject_injury'], MN_police_data['force_type'])
crossing

force_type,Baton,Bodily Force,Chemical Irritant,Firearm,Gun Point Display,Improvised Weapon,Less Lethal,Less Lethal Projectile,Maximal Restraint Technique,Police K9 Bite,Taser
subject_injury,,,,,,,,,,,
No,0,1093,131,2,33,34,0,1,0,2,150
Not recorded,2,7051,1421,0,27,74,87,0,170,31,985
Yes,2,1286,41,0,44,40,0,2,0,44,172


The missing data appears to have a high percentage of the bodily force and chemical irritant force types. It is possible that the missing subject injury cases with those force types resulted in an injury, but just not one that necessarily needed to be reported. What is likely is that a fair amount of the missing subject injuries are injuries that the subjects wouldn't report so the police didn't feel the need to.

**Dummy Variable**

In [29]:
from google.colab import files

uploaded = files.upload()

Saving justice_data.parquet to justice_data.parquet


In [51]:
justice_data = pd.read_parquet('justice_data.parquet')
justice_data.head()

,InternalStudyID,REQ_REC#,Defendant_Sex,Defendant_Race,Defendant_BirthYear,Defendant_Age,Defendant_AgeGroup,Defendant_AgeatCurrentArrest,Defendant_AttorneyTypeAtCaseClosure,Defendant_IndigencyStatus,...,NewFelonySexualAssaultArrest_OffDate,NewFelonySexualAssaultArrest_ArrestDate,NewFelonySexualAssaultArrest_DaysBetweenContactEventandOffDate,NewFelonySexualAssaultArrest_DaysBetweenOffDateandArrestDate,NewFelonySexualAssaultArrest_DaysBetweenReleaseDateandOffDate,NewFelonySexualAssaultArrest_Disposition,Intertnalindicator_ReasonforExcludingFromFollowUpAnalysis,CriminalHistoryRecordsReturnedorCMSRecordsFoundforIndividual,DispRecordFoundforChargesinOct2017Contact_Atleast1dispfound,CrimeCommission2021ReportClassificationofDefendants
0,ADI00001,1,M,W,1986,31,3,31,99,99,...,,,,999,999,,4,1,0,Defendant could not be classified or tracked d...
1,ADI00007,3,M,B,1956,60,6,60,9,9,...,,,,999,999,,5,1,1,Defendant Detained Entire Pre-Trial Period_Und...
2,ADI00008,4,M,W,1990,27,3,27,9,9,...,,,,999,999,,5,1,1,Defendant Detained Entire Pre-Trial Period_Und...
3,CDI00036,6,M,B,1989,27,3,27,0,0,...,,,,999,999,,5,1,1,Defendant Detained Entire Pre-Trial Period_Und...
4,CDI00038,7,F,W,1988,28,3,28,0,0,...,,,,999,999,,0,1,1,New criminal offense punishable by incarcerati...


In [52]:
justice_data['WhetherDefendantWasReleasedPretrial'].isnull().sum()

0

In [53]:
justice_data['WhetherDefendantWasReleasedPretrial'].unique()

array([9, 0, 1])

In [54]:
justice_data['WhetherDefendantWasReleasedPretrial'].describe()

,WhetherDefendantWasReleasedPretrial
count,22986.000000
mean,0.845428
std,0.477276
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,9.000000


In [55]:
print(justice_data['WhetherDefendantWasReleasedPretrial'].value_counts())

WhetherDefendantWasReleasedPretrial
1    19154
0     3801
9       31
Name: count, dtype: int64


In [56]:
cross_pretrial = pd.crosstab((justice_data['WhetherDefendantWasReleasedPretrial']==9), justice_data['PretrialReleaseDate'])
cross_pretrial

PretrialReleaseDate,,1/10/2018,1/11/2018,1/12/2018,1/16/2018,1/17/2018,1/18/2018,1/19/2018,1/20/2018,1/22/2018,...,6/28/2018,7/25/2018,7/26/2018,7/3/2018,7/31/2018,8/23/2018,8/26/2018,8/27/2018,8/29/2018,8/9/2018
WhetherDefendantWasReleasedPretrial,,,,,,,,,,,,,,,,,,,,,
False,3952,7,4,1,3,3,3,6,1,8,...,1,2,1,1,1,1,1,2,1,1
True,31,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
cross_bond = pd.crosstab(justice_data['WhetherDefendantWasReleasedPretrial'], justice_data['BondTypeAtInitialContact'])
cross_bond

BondTypeAtInitialContact,1,2,3,4,5,6,7,9
WhetherDefendantWasReleasedPretrial,,,,,,,,
0,3368,360,7,2,21,0,39,4
1,2575,3776,5411,1802,233,5285,72,0
9,10,13,1,1,0,0,6,0


In [59]:
#we are going to replace some of the 9s (as unknown if released pretrial) based on if they were given a bond pretrial or denyed one
justice_data.loc[justice_data['BondTypeAtInitialContact'] == 1, 'WhetherDefendantWasReleasedPretrial'] = 0
#this essentially means that if the person wasn't given a bond at initial contact we are assuming that they were not released, which is an assumption but a valid one (in my opinion)

In [60]:
print(justice_data['WhetherDefendantWasReleasedPretrial'].value_counts())

WhetherDefendantWasReleasedPretrial
1    16579
0     6386
9       21
Name: count, dtype: int64


In [63]:
#we are making the other 9s NaNs so that it is clear they are missing values
justice_data['WhetherDefendantWasReleasedPretrial'] = justice_data['WhetherDefendantWasReleasedPretrial'].replace(9, np.nan)
print(justice_data['WhetherDefendantWasReleasedPretrial'].value_counts())

WhetherDefendantWasReleasedPretrial
1.0    16579
0.0     6386
Name: count, dtype: int64


**Missing Values, not at Random**

In [66]:
justice_data['ImposedSentenceAllChargeInContactEvent'].value_counts()

,count
ImposedSentenceAllChargeInContactEvent,
,9053
0,4953
12,1404
.985626283367556,1051
6,809
...,...
49.9712525667351,1
57.0349075975359,1
79.9260780287474,1


**Q2.** Go to https://sharkattackfile.net/ and download their dataset on shark attacks.

1. Open the shark attack file using Pandas. It is probably not a csv file, so `read_csv` won't work.
2. Drop any columns that do not contain data.
3. Clean the year variable. Describe the range of values you see. Filter the rows to focus on attacks since 1940. Are attacks increasing, decreasing, or remaining constant over time?
4. Clean the Age variable and make a histogram of the ages of the victims.
5. What proportion of victims are male?
6. Clean the `Type` variable so it only takes three values: Provoked and Unprovoked and Unknown. What proportion of attacks are unprovoked?
7. Clean the `Fatal Y/N` variable so it only takes three values: Y, N, and Unknown.
8. Are sharks more likely to launch unprovoked attacks on men or women? Is the attack more or less likely to be fatal when the attack is provoked or unprovoked? Is it more or less likely to be fatal when the victim is male or female? How do you feel about sharks?
9. What proportion of attacks appear to be by white sharks? (Hint: `str.split()` makes a vector of text values into a list of lists, split by spaces.)

**Q3.** Open the "tidy_data.pdf" document in the repo, which is a paper called Tidy Data by Hadley Wickham.

  1. Read the abstract. What is this paper about?
  2. Read the introduction. What is the "tidy data standard" intended to accomplish?
  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."
  4. Read Section 2.2. How does Wickham define values, variables, and observations?
  5. How is "Tidy Data" defined in section 2.3?
  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?
  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?
  8. Read Section 6. What is the "chicken-and-egg" problem with focusing on tidy data? What does Wickham hope happens in the future with further work on the subject of data wrangling?

**Q4.** Many important datasets contain a race variable, typically limited to a handful of values often including Black, White, Asian, Latino, and Indigenous. This question looks at data gathering efforts on this variable by the U.S. Federal government.

1. How did the most recent US Census gather data on race?
2. Why do we gather these data? What role do these kinds of data play in politics and society? Why does data quality matter?
3. Please provide a constructive criticism of how the Census was conducted: What was done well? What do you think was missing? How should future large scale surveys be adjusted to best reflect the diversity of the population? Could some of the Census' good practices be adopted more widely to gather richer and more useful data?
4. How did the Census gather data on sex and gender? Please provide a similar constructive criticism of their practices.
5. When it comes to cleaning data, what concerns do you have about protected characteristics like sex, gender, sexual identity, or race? What challenges can you imagine arising when there are missing values? What good or bad practices might people adopt, and why?
6. Suppose someone invented an algorithm to impute values for protected characteristics like race, gender, sex, or sexuality. What kinds of concerns would you have?